# Chapter 5 code for Julia for Machine Learning book

(c) 2020 by Zacharias Voulgaris, all rights reserved.

This code has been tested for Julia 1.4.1. 

# Initialization

## Packages

Get these packages in memory for the experiments that follow.

In [3]:
using CSV, DataFrames;

In [ ]:
using Clustering, DecisionTree, GLM, XGBoost;

In [ ]:
using StatsBase, Distributions, HypothesisTests, MultivariateStats;

In [ ]:
using Distances, MLLabelUtils, MLBase;

In [ ]:
using TSne, Gadfly;

In [ ]:
using ScikitLearn.CrossValidation: cross_val_score;

In [ ]:
using Random;

Note: if you got an alarming large area of red in the outputs of these commands, don't panic. Most of these are just warnings that most likely won't affect the experience of using these packages. Also, the first time you run these packages it is bound to take more time than other times.
 

## Auxiliary functions

In [ ]:
function partition(ind::Union{Array{Int64, 1}, UnitRange{Int64}}, r::Float64, shuffle::Bool = true)
    if typeof(ind) == UnitRange{Int64}; ind = collect(ind); end
    N = length(ind) # total number of data points in sample
    n = round(Int64, N*r) # number of data points in training set (train)
    train = [0, 0] # initialize train output
    test = [0, 0] # initialize test output
    
    if shuffle        
        ind_ = ind[randperm(N)]
    else
        ind_ = ind
    end
    
    train = ind_[1:n]
    test = ind_[(n+1):end]
    return train, test
end

# Load data from localization.csv file

In [4]:
df = CSV.read("localization.csv", header = false);

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/xronos/.julia/packages/CSV/OM6FO/src/CSV.jl:40


In [5]:
old_names = names(df)

8-element Array{String,1}:
 "Column1"
 "Column2"
 "Column3"
 "Column4"
 "Column5"
 "Column6"
 "Column7"
 "Column8"

In [6]:
new_names = map(Symbol, ["WiFi1", "WiFi2", "WiFi3", "WiFi4", "WiFi5", "WiFi6", "WiFi7", "Room"])

8-element Array{Symbol,1}:
 :WiFi1
 :WiFi2
 :WiFi3
 :WiFi4
 :WiFi5
 :WiFi6
 :WiFi7
 :Room

In [7]:
new_names

8-element Array{Symbol,1}:
 :WiFi1
 :WiFi2
 :WiFi3
 :WiFi4
 :WiFi5
 :WiFi6
 :WiFi7
 :Room

In [8]:
for i = 1:8
    rename!(df, old_names[i] => new_names[i])
end

In [9]:
names(df)

8-element Array{String,1}:
 "WiFi1"
 "WiFi2"
 "WiFi3"
 "WiFi4"
 "WiFi5"
 "WiFi6"
 "WiFi7"
 "Room"

In [10]:
df[:RegressionTarget] = Matrix(df[[1, 4, 6, 7]]) * [5, 10, 15, 20] + 0.01*randn(2000)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[10]:1
└ @ Core In[10]:1
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[10]:1
└ @ Core In[10]:1


2000-element Array{Float64,1}:
 -3830.002631972675
 -3965.010884458538
 -3939.9928327901084
 -3875.0082832577796
 -3899.9936876556308
 -3959.9986051486585
 -3980.011003980739
 -3909.9887182064867
 -3884.988397524366
 -4099.999810322275
 -4069.995386146227
 -3905.008865541476
 -3904.980101474007
     ⋮
 -3815.002347675954
 -3655.004752396686
 -3779.9956609616597
 -3719.99371191653
 -3969.9905288490754
 -4009.994156410997
 -4000.0029350906543
 -4124.994822336597
 -4020.0100426615486
 -4025.0240927191053
 -3969.9920797881955
 -3954.9990074906914

In [ ]:
first(df, 10)

# Work the data using StatsBase package

In [ ]:
XX = map(Float64, Matrix(df[1:7]));

In [ ]:
X = StatsBase.standardize(ZScoreTransform, XX, dims=2)

In [ ]:
df2 = DataFrame(hcat(X, df[:Room]))

In [ ]:
for i = 1:8
    rename!(df2, names(df2)[i] => new_names[i])
end

In [ ]:
first(df2, 10)

Note: the localization dataset is used as an example as it's fairly straight-forward and doesn't require pretty much any data engineering. Feel free to experiment with different datasets, for your predictive analytics projects.
 

# Work the data using Clustering package

In [ ]:
nc = 4; # number of clusters

In [ ]:
n, m = size(X)
XX = Array{Float64}(undef, m, n);

for i = 1:m
    XX[i,:] = X[:,i]
end

# alternatively, if you use the LinearAlgebra package, you can do the same thing as follows: XX = collect(adjoint(X))

In [ ]:
R = kmeans(XX, nc; maxiter=200, display=:iter) # the second part of the arguments is optional but useful in a demo

In [ ]:
@assert nclusters(R) == nc # although it's pretty obvious from the previous output that the number of clusters is indeed 4, it doesn't hurt to verify it

In [ ]:
a = assignments(R) # assignments (labels) of the various points of the datasets, according to the Kmeans model

In [ ]:
c = counts(R) # Cluster sizes

In [ ]:
M = (R.centers)' # cluster centers (centroids), transposed for easier viewing

In [ ]:
R2 = fuzzy_cmeans(XX, nc, 2, maxiter=200, display=:iter) # 2 is the fuzziness parameter (needs to be > 1)

In [ ]:
M2 = (R2.centers)' # the centroids, transposed for easier viewing

In [ ]:
memberships = R2.weights # how much each data point belongs to each one of the (4) clusters according to Cmeans

# Work the data using predictive ML models

## Prepare data for ML models

In [ ]:
X = df2[1:7];

In [ ]:
XX = map(Float64, Matrix(X));

In [ ]:
y1 = map(string, df[:Room]);

In [ ]:
y2 = df[end];

In [ ]:
train, test = partition(1:n, 0.7, true) # 70-30 split of the data

## Build and run the ML models

### Decision Tree

In [ ]:
tree = DecisionTreeClassifier(max_depth=3);

In [ ]:
DecisionTree.fit!(tree, XX[train,:], y1[train])

In [ ]:
print_tree(tree)

In [ ]:
yhat = DecisionTree.predict(tree, XX[test,:])

In [ ]:
sum(yhat .== y1[test]) / length(test)

In [ ]:
accuracy = cross_val_score(tree, XX, y1, cv=5)

### Random Forests

In [ ]:
n_subfeatures = 3;
n_trees = 10;
pst = 0.5; # portion of samples per tree
max_depth = 6;
n_folds = 5; # number of folds for the K-fold cross-validation later on

In [ ]:
forest1 = build_forest(y1[train], XX[train,:], n_subfeatures, n_trees, pst, max_depth);

In [ ]:
yhat = apply_forest(forest1, XX[test,:])

In [ ]:
scores = apply_forest_proba(forest1, XX[test,:], map(string, 1:4)) # map(string, 1:4) => ["1", "2", "3", "4"] but it's more elegant and less risky

In [ ]:
accuracy = nfoldCV_forest(y1, XX, n_folds, n_subfeatures)

In [ ]:
forest2 = build_forest(y2[train], XX[train,:], 2, 10, 0.5, 6);

In [ ]:
apply_forest(forest2, XX[test,:])

In [ ]:
accuracy = nfoldCV_forest(y2, XX, n_folds, n_subfeatures)

In [ ]:
ni = 10 # number of iterations for boosted stumps

In [ ]:
model, coeffs = build_adaboost_stumps(y1[train], XX[train,:], ni)

In [ ]:
apply_adaboost_stumps(model, coeffs, XX[test,:])

In [ ]:
apply_adaboost_stumps_proba(model, coeffs, XX[test,:], map(string, 1:4))

In [ ]:
accuracy = nfoldCV_stumps(y1, XX, n_folds, ni)

If this isn't an overfit model, I don't know what is!
 

# ML utilities packages

## MLBase functions

For the love of God, don't bother with the label functions of this package as they are a nightmare to figure out and do something useful with them! Fortunately, there is the MLLabelsUtils package for this (see next section).

### Preprocessing target and prediction variables

In [ ]:
pred = map(x -> parse(Int64, x), yhat); # prediction of RF classifier

In [ ]:
gt = map(x -> parse(Int64, x), y1[test]); # ground truth for classification

In [ ]:
conf = maximum(scores, dims=2)[:]; # confidence scores for RF classifier

In [ ]:
k = length(unique(gt)); # number of classes

### Performance evaluation functions for classification

In [ ]:
correctrate(gt, pred) # accuracy rate

In [ ]:
confusmat(k, gt, pred) # confusion matrix for the RF classifier

In [ ]:
ROC =  roc(gt, conf) # Receiver Operator Characteristic data (for building an ROC curve) for RFC

In [ ]:
r = ROC[50] # a particular instance of the ROC data array

In [ ]:
precision(r)

In [ ]:
recall(r)

In [ ]:
f1score(r)

### Crossvalidation functions

In [ ]:
n = length(y1) # total number of data points
k = round(Int64, 0.3*n) # size of a sample for test set
sn = 5 # number of samples

In [ ]:
Kfold(n, k)

In [ ]:
LOOCV(n)

In [ ]:
RandomSub(n, sn, k)

## MLLabelsUtils functions

In [ ]:
y1[rand(1:n, 20)] # 20 random labels just to remind ourselves of what the classification target variable is like

In [ ]:
labelenc(y1)

In [ ]:
islabelenc(y1, LabelEnc.ZeroOne)

In [ ]:
convertlabel([:Room1,:Room2,:Room3,:Room4], y1)

In [ ]:
convertlabel(LabelEnc.Indices{Int}, y1) 

In [ ]:
convertlabel(LabelEnc.OneOfK{Bool}, y1)

In [ ]:
convertlabel(LabelEnc.OneOfK{Float64}, y1)

In [ ]:
convertlabel(LabelEnc.TrueFalse, y1, LabelEnc.OneVsRest("1"))

In [ ]:
scores[1:10,:] # confidence scores for the RF classifier

In [ ]:
MLLabelUtils.classify(scores[1:10,:]', LabelEnc.OneOfK) # note: the scores are transposed using the ' operator

# Make Use of the "Other" Packages for Some Cool Plots

## Distances package

In [ ]:
D = [Euclidean(), Cityblock(), Jaccard(), CosineDist(), CorrDist()];

In [ ]:
y = map(Float64, y2);

In [ ]:
for dist in D
    println(Distances.evaluate(dist, X[:,1], y2))
end

In [ ]:
CosineDist()(X[:,1], y2)

In [ ]:
D = pairwise(Jaccard(), XX, dims=2)

Note that the pairwise function works only with matrices, hence the use of XX here instead of X (X is a dataframe)

## Gadfly package

In [ ]:
plot(df, x = :WiFi1, y = :RegressionTarget)

Clearly there is a trend here, showing that the RegressionTarget variable is related to the WiFi1 variable

In [ ]:
plot(df2, x = :WiFi1, y = :WiFi2, color = :Room)

In [ ]:
plot(df3, x = :x1, y = :x2, color = :x3)

In [ ]:
plot(df2, x = "WiFi1", Geom.histogram)

In [ ]:
plot(df2, x = "WiFi1", Geom.histogram(bincount=30))

In [ ]:
plot(df2, x=:WiFi1, y=:WiFi2, Geom.histogram2d(xbincount=40, ybincount=40))

In [ ]:
spy(D) # heatmap of the Jaccard distance matrix from a previous section

# Sandbox

A place for you to experiment with the above material, without making a mess of the whole notebook or disrupt its flow. It's much easier to maintain this section than keep track of various copies of this notebook.